In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text

In [1]:
def model_scores_comparator(add_time,add_posts,bert,stock,l):
    '''
    Args:
    add_time: address of the file that has time series data of the stock
    add_posts: address of the  file that has all the posts of the stock, labelled as spam or not spam
    bert: address of the bert sentiment analysis model
    stock: name of the stock
    l: list of all models that the target model's results have to be compared to

    Returns:
    A pandas dataframe containing the MAEs and MSEs of all the models
    '''
    import pandas as pd
    import tensorflow as tf
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import tensorflow_hub as hub
    import tensorflow_text as text
    df_stock = pd.read_csv(add_time,
                     parse_dates = ["Date"],
                     index_col = ["Date"])
    df_posts = pd.read_excel(add_posts)
    # removing the unnecessary columns
    df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
    # removing spam posts
    df_posts = df_posts[df_posts.Spam==0.0]
    df_posts.drop(["Spam"],axis=1,inplace=True)
    # sliding a window of 7 days and adding all the TIs
    from stock_helper import prepare_data
    x,y = prepare_data(df_stock)
    # slicing the data
    final_x = x[np.datetime64("2021-11-13"):]
    final_y = y[np.datetime64("2021-11-13"):]
    # reversing the posts data
    df_posts = df_posts[::-1]
    final_posts = df_posts[7:]
    # loading the sentiment analysis model
    sent_model = tf.keras.models.load_model(bert)
    # removing duplicates from the data
    final_posts.drop_duplicates(subset=['Messages'])
    # calculating the sentiments score
    sentiments = []
    prev = np.datetime64("2015-11-12 21:31:26")
    for i in final_y.index:
        total=0
        cnt=0
        for j in final_posts.itertuples():
            _,msg,time = j
            if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
                total += tf.squeeze(sent_model.predict([msg])).numpy()
                cnt+=1
        prev = np.datetime64(i)
        if(cnt==0):
            sentiments.append(0)
        else:
            sentiments.append(total/cnt)
    # getting indices where sentiments score is 0
    zero_index = []
    for i,j in enumerate(sentiments):
        if(j==0):
            zero_index.append(i)
    # removing all the zero values indices
    sentiments = np.delete(sentiments,zero_index)
    final_x_zeros = final_x.copy()
    final_y_zeros = final_y.copy()
    final_y_zeros = final_y_zeros.to_frame()
    final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
    final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
    final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
    final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
    # removing the added helper column
    final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
    final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)
    # preparing the scaled data
    from sklearn.preprocessing import MinMaxScaler
    mms = MinMaxScaler()
    final_x_zeros_scaled = mms.fit_transform(final_x_zeros)
    max_val = final_y_zeros.max()
    final_y_zeros_scaled = final_y_zeros/max_val
    # loading the nbeats model for the given stock
    model_nbeats = tf.keras.models.load_model("nbeats_" + stock)
    # making predictions and building the final dataframe 
    preds = tf.squeeze(model_nbeats.predict(final_x_zeros)).numpy()
    final_df = pd.DataFrame({"Predictions": preds,
                        "Sentiments":sentiments})
    # building and training the target model
    tf.random.set_seed(42)
    inputs = tf.keras.Input(shape=(2))
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(inputs)
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
    x = tf.keras.layers.Dense(8, activation="selu", kernel_initializer="lecun_normal")(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.models.Model(inputs=inputs,outputs=outputs)
    
    k = int(np.round(len(final_x_zeros)*0.8))
    xtrain,ytrain,xtest,ytest = final_df[:k],final_y_zeros[:k],final_df[k:],final_y_zeros[k:]
    
    model.compile(loss = "mae", optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
    history = model.fit(xtrain,ytrain,
                        epochs = 1000,
                        validation_data=(xtest,ytest), 
                        verbose=0,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                                patience=200, 
                                                                restore_best_weights=True),
                              tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                                   patience=100, 
                                                                   verbose=1)])
    target_result = model.evaluate(xtest,ytest)
    dict_res = {
        "Target_model":[target_result[1]]
    }
    df = pd.DataFrame(dict_res)
    frames = []
    frames.append(df)
    # getting the results of all the other models
    scaled_models = ["GRU","LSTM"]
    for i in l:
        name = i + "_" + stock
        model = tf.keras.models.load_model(name)
        if i in scaled_models:
            print("scaled_model")
            preds = tf.squeeze(model.predict(final_x_zeros_scaled[k:])).numpy()
            preds = preds*max_val.values[0]
            print(preds)
            result = sum(abs(preds-tf.squeeze(final_y_zeros[k:].to_numpy()).numpy()))
            print(tf.squeeze(final_y_zeros[k:].to_numpy()).numpy())
            dict_res = {
             i:[result]
            }
            df = pd.DataFrame(dict_res)
            frames.append(df)
        else:
            result = model.evaluate(final_x_zeros[k:], final_y_zeros[k:])
            dict_res = {
             i:[result[1]]
            }
            df = pd.DataFrame(dict_res)
            frames.append(df)
    return pd.concat(frames, axis=1)

In [3]:
df = model_scores_comparator("tata_steel/TATASTEEL.NS (1).csv", "tata_steel/tata_steel_final_posts.xlsx", 
                             "final_bert", "tata steel", ["conv1d","LSTM","GRU"])
df


Epoch 00158: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00258: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 2s 2s/step - loss: 48.7430 - mae: 48.7430 - mse: 2729.7170
scaled_model
[1135.9811 1127.7804 1116.2896 1110.5485 1146.7969 1164.5603 1167.7561
 1175.9675 1182.9775 1194.5582 1203.2809 1206.635  1209.3304]
[1109.1  1088.35 1084.65 1085.55 1167.35 1167.5  1166.6  1176.3  1183.
 1219.6  1222.35 1248.35 1254.45]
scaled_model
[1135.5529 1122.5474 1110.4327 1087.9885 1145.4143 1140.6273 1135.6115
 1149.174  1174.2701 1215.2115 1233.9827 1245.7291 1248.0344]
[1109.1  1088.35 1084.65 1085.55 1167.35 1167.5  1166.6  1176.3  1183.
 1219.6  1222.35 1248.35 1254.45]


,Target_model,conv1d,LSTM,GRU
0,26.981972,48.742985,278.898682,229.581787


In [4]:
df = model_scores_comparator("icici/ICICIBANK.NS.csv", "icici/icici_bank_final_posts.xlsx", 
                             "final_bert", "icici", ["conv1d","LSTM","GRU"])
df


Epoch 00155: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00255: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 0s 480ms/step - loss: 16.6612 - mae: 16.6612 - mse: 479.5851
scaled_model
[802.32715 800.06696 794.68695 791.75793 791.4672  793.3288  794.8532
 794.66565 787.3845  789.2938  792.50085]
[801.65 794.65 781.15 788.8  810.3  813.75 808.95 805.05 792.5  802.8
 790.8 ]
scaled_model
[794.7976  796.802   789.7172  794.16956 808.69275 816.62695 805.85
 797.77545 763.6633  782.0082  791.0225 ]
[801.65 794.65 781.15 788.8  810.3  813.75 808.95 805.05 792.5  802.8
 790.8 ]


,Target_model,conv1d,LSTM,GRU
0,10.606112,16.661226,106.646545,87.65094


In [5]:
df = model_scores_comparator("itc/ITC.NS.csv", "itc/itc_final_posts.xlsx", 
                             "final_bert", "itc", ["conv1d","LSTM","GRU"])
df


Epoch 00156: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00256: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 0s 416ms/step - loss: 4.3033 - mae: 4.3033 - mse: 40.9895
scaled_model
[214.65805 216.33624 219.05553 225.66968 230.60774 233.77744 232.622
 231.33575 230.19392 229.92274 230.17053]
[214.6  217.6  220.2  227.75 232.15 234.45 230.2  231.25 230.15 232.25
 232.45]
scaled_model
[215.96233 217.58995 218.2951  226.66629 229.85011 232.40779 229.33708
 231.0536  230.60959 231.98804 231.61888]
[214.6  217.6  220.2  227.75 232.15 234.45 230.2  231.25 230.15 232.25
 232.45]


,Target_model,conv1d,LSTM,GRU
0,2.929909,4.303325,13.919815,11.315048


In [8]:
df = model_scores_comparator("maruti_suzuki/MARUTI.NS.csv", "maruti_suzuki/maruti_suzuki_final_posts.xlsx", 
                             "final_bert", "maruti suzuki", ["conv1d","LSTM","GRU"])
df


Epoch 00155: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00255: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
1/1 [==============================] - 1s 695ms/step - loss: 113.8571 - mae: 113.8571 - mse: 27533.9277
scaled_model
[8506.652 8530.543 8552.735 8562.858 8570.454 8602.898 8630.671 8642.096
 8650.431 8694.767 8715.952 8726.964]
[8820.2  8550.95 8597.3  8559.4  8515.25 8593.65 8550.55 8511.65 8599.45
 8949.45 8805.45 8737.15]
scaled_model
[8766.409 8624.287 8584.201 8574.237 8517.011 8597.31  8564.397 8577.385
 8662.956 8836.088 8747.038 8682.091]
[8820.2  8550.95 8597.3  8559.4  8515.25 8593.65 8550.55 8511.65 8599.45
 8949.45 8805.45 8737.15]


,Target_model,conv1d,LSTM,GRU
0,96.009926,113.857094,1062.344727,530.405273
